In [549]:
import torch
from scipy.integrate import quad

In [417]:
H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.double)
M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.double)
sig = torch.tensor([[[0.5249],[0.4072]]], dtype=torch.double) 
C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive definite
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
T = 5.0

In [419]:
t_batch = torch.linspace(0.2,0.3,20)

T = 1
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite

In [494]:
x_batch = torch.ones([t_batch.shape[0],1,2], dtype=torch.double)

In [423]:
x_batch

tensor([[[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]],

        [[1., 1.]]])

In [425]:
t_batch.shape[0]

20

In [427]:
time_grids = time_grids_tensor = torch.stack([
    torch.linspace(0, T, 6, dtype=torch.double) for i in [0]
])

In [429]:
time_grids[:,0]

tensor([0.], dtype=torch.float64)

In [431]:
import numpy as np
time_grids[:,0]>=0

tensor([True])

In [433]:
(torch.all(np.abs(time_grids[:,-1] - T) <= 1e-12) or torch.all(time_grids[:,0]>=0))

tensor(True)

In [435]:
time_grids_in = torch.flip(time_grids, dims=[1])

In [437]:
time_grids_in

tensor([[1.0000, 0.8000, 0.6000, 0.4000, 0.2000, 0.0000]], dtype=torch.float64)

In [439]:
S = R.clone()

In [441]:
repl = torch.ones(time_grids_in.shape)

In [443]:
repl

tensor([[1., 1., 1., 1., 1., 1.]])

In [565]:
from concurrent.futures import ProcessPoolExecutor

In [445]:
rep_exd = repl.unsqueeze(-1).unsqueeze(-1)  # 现在形状为 (a, b, 1, 1)

# 将矩阵扩展到 (1, 1, m, n) 以便进行广播
S_exd = S.unsqueeze(0).unsqueeze(0) 

S_repl = rep_exd*S_exd
dt = time_grids_in[:,1:]-time_grids_in[:,:-1]

In [447]:
S_repl = torch.flip(S_repl, dims=[1])

In [449]:
S_repl

tensor([[[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]]], dtype=torch.float64)

In [451]:
def euler_step(S_in, dt_in):
    
    dS_in = -2 * H.T @ S_in + S_in @ M @ torch.inverse(D) @ M.T @ S_in - C

    dt_resized = dt_in[:, None, None]

    return S_in + dS_in * dt_resized

def rk4_step(S_in, dt_in):
    def riccati_derivative(S):
        return -2 * H.T @ S_in + S_in @ M @ torch.inverse(D) @ M.T @ S_in - C
    dt_resized = dt_in[:, None, None]
    k1 = riccati_derivative(S_in)
    k2 = riccati_derivative(S_in + 0.5 *  k1 * dt_resized)
    k3 = riccati_derivative(S_in + 0.5 *  k2 * dt_resized)
    k4 = riccati_derivative(S_in + k3 * dt_resized)

    return S_in + (k1 + 2*k2 + 2*k3 + k4)*dt_resized/6

for i in range(dt.shape[1]):
    
    S_repl[:,i+1] = euler_step(S_repl[:,i], dt[:,i])

    
    #S_repl[:,i+1] = rk4_step(S_repl[:,i], dt[:,i])


S_repl = torch.flip(S_repl, dims=[1])

In [453]:
index_s_1 = torch.searchsorted(time_grids[0,:], torch.min(t_batch), right=True) - 1

In [455]:
time_grids[0,index_s_1:]

tensor([0.2000, 0.4000, 0.6000, 0.8000, 1.0000], dtype=torch.float64)

In [457]:
time_grid_for_intpl = time_grids[0,index_s_1:]
S_repl_for_intpl = S_repl[0,index_s_1:]

In [459]:
time_grid_for_intpl.dtype

torch.float64

In [461]:
index_s_1 

tensor(1)

In [536]:
S_repl_for_intpl

tensor([[[ 1.5008, -0.5171],
         [ 0.8353, -0.1473]],

        [[ 1.5552, -0.6097],
         [ 0.8742, -0.2166]],

        [[ 1.6160, -0.7360],
         [ 0.8960, -0.2800]],

        [[ 1.6000, -0.8000],
         [ 0.8000, -0.2000]],

        [[ 1.0000,  0.0000],
         [ 0.0000,  1.0000]]], dtype=torch.float64)

In [540]:
traces = torch.einsum('bii->b', sig @ sig.transpose(1,2) @ S_repl_for_intpl).unsqueeze(1).unsqueeze(2)

Process SpawnProcess-2:
Process SpawnProcess-6:
Process SpawnProcess-11:
Process SpawnProcess-12:
Process SpawnProcess-8:
Process SpawnProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-1:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-7:
Process SpawnProcess-4:
Process SpawnProcess-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ysrae1/anaconda3/lib

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [546]:
trace_cubic = CubicSpline(time_grid_np, traces.numpy())

In [559]:
def int_fun(t):
    
    return trace_cubic(t)

In [569]:
trace_cubic

In [561]:
result, error = quad(int_fun, 0.2, T)

In [571]:
t_batch_np

array([0.2       , 0.20526317, 0.21052632, 0.21578948, 0.22105263,
       0.2263158 , 0.23157896, 0.23684211, 0.24210528, 0.24736843,
       0.25263157, 0.25789475, 0.2631579 , 0.26842105, 0.27368423,
       0.27894738, 0.28421053, 0.28947368, 0.29473686, 0.3       ],
      dtype=float32)

In [599]:
for i in range(len(t_batch_np)):
    intgl, error = quad(int_fun, t_batch_np[i], T)
    xTSx[i,0]+= intgl

In [597]:
xTSx[1,0]

tensor([1.6695], dtype=torch.float64)

In [ ]:
S_repl_for_intpl

In [463]:
from scipy.interpolate import CubicSpline
# 将PyTorch张量转换为NumPy数组
time_grid_np = time_grid_for_intpl.numpy()
S_repl_for_intpl_np = S_repl_for_intpl.numpy()
t_batch_np = t_batch.numpy()



# 创建三次样条插值函数
cubic_spline = CubicSpline(time_grid_np, S_repl_for_intpl_np)

In [499]:
def S_intpl(t_batch_np_in):
    return torch.from_numpy(cubic_spline(t_batch_np_in))

S_t_s = S_intpl(t_batch_np)
x_batch_T = x_batch.transpose(1,2)
xTSx = x_batch @ S_t_s @ x_batch_T

In [581]:
xTSx[0,0]+= 

tensor([3.6717], dtype=torch.float64)

In [519]:
def traces(t_batch_np_in):
    S_t_hat = S_intpl(t_batch_np_in)
    return torch.einsum('bii->b', sig @ sig.transpose(1,2) @ S_t_hat.transpose(1,2)).unsqueeze(1).unsqueeze(2)

In [521]:
traces(t_batch_np)

tensor([[[0.4571]],

        [[0.4568]],

        [[0.4566]],

        [[0.4563]],

        [[0.4561]],

        [[0.4558]],

        [[0.4556]],

        [[0.4554]],

        [[0.4552]],

        [[0.4549]],

        [[0.4547]],

        [[0.4545]],

        [[0.4543]],

        [[0.4541]],

        [[0.4539]],

        [[0.4537]],

        [[0.4535]],

        [[0.4533]],

        [[0.4531]],

        [[0.4529]]], dtype=torch.float64)

In [ ]:
traces = torch.einsum('bcii->bc', sig @ sig.transpose(1,2) @ S_t_T.transpose(0,1)).unsqueeze(1).unsqueeze(2)

In [524]:
integral_value, error = torch.quad(traces, t_batch_np, T)

AttributeError: module 'torch' has no attribute 'quad'

In [501]:
xTSx

tensor([[[1.6717]],

        [[1.6695]],

        [[1.6673]],

        [[1.6651]],

        [[1.6630]],

        [[1.6610]],

        [[1.6590]],

        [[1.6570]],

        [[1.6550]],

        [[1.6531]],

        [[1.6512]],

        [[1.6494]],

        [[1.6475]],

        [[1.6457]],

        [[1.6440]],

        [[1.6422]],

        [[1.6405]],

        [[1.6387]],

        [[1.6370]],

        [[1.6353]]], dtype=torch.float64)

In [492]:
x_batch.dtype

torch.float32

In [482]:
cubic_spline(t_batch_np)

array([[[ 1.50080527, -0.51712491],
        [ 0.83528246, -0.14725131]],

       [[ 1.50234677, -0.51981014],
        [ 0.8366994 , -0.14976123]],

       [[ 1.50387733, -0.52246434],
        [ 0.83809057, -0.15221829]],

       [[ 1.50539738, -0.525089  ],
        [ 0.83945636, -0.15462383]],

       [[ 1.50690732, -0.52768556],
        [ 0.84079713, -0.15697914]],

       [[ 1.50840757, -0.53025552],
        [ 0.84211327, -0.15928557]],

       [[ 1.50989856, -0.53280032],
        [ 0.84340515, -0.16154442]],

       [[ 1.51138068, -0.53532144],
        [ 0.84467315, -0.16375699]],

       [[ 1.51285437, -0.53782036],
        [ 0.84591764, -0.16592463]],

       [[ 1.51432002, -0.54029852],
        [ 0.847139  , -0.16804863]],

       [[ 1.51577807, -0.54275741],
        [ 0.84833761, -0.17013033]],

       [[ 1.51722893, -0.54519851],
        [ 0.84951384, -0.17217104]],

       [[ 1.518673  , -0.54762325],
        [ 0.85066807, -0.17417207]],

       [[ 1.5201107 , -0.55003311],
  

In [467]:
torch.from_numpy(cubic_spline(0.2))

tensor([[ 1.5008, -0.5171],
        [ 0.8353, -0.1473]], dtype=torch.float64)

In [469]:
S_repl[0,2:]

tensor([[[ 1.5552, -0.6097],
         [ 0.8742, -0.2166]],

        [[ 1.6160, -0.7360],
         [ 0.8960, -0.2800]],

        [[ 1.6000, -0.8000],
         [ 0.8000, -0.2000]],

        [[ 1.0000,  0.0000],
         [ 0.0000,  1.0000]]], dtype=torch.float64)

In [471]:
x_trans = x_batch.transpose(1,2)

In [473]:
x_batch@x_batch.transpose(1,2)

tensor([[[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]],

        [[2.]]])

In [475]:
S_t_s = S_repl[:,0,:,:]
S_t_T = S_repl[:,1:,:,:]

In [ ]:
S_t_s = S_repl[:,0,:,:]

In [484]:
S_t_s

tensor([[[ 1.4688, -0.4662],
         [ 0.8075, -0.1022]]], dtype=torch.float64)

In [477]:
print(sig.shape)
print(sig.transpose(1,2).shape)
print(S_t_T.transpose(0,1).shape)

torch.Size([1, 2, 1])
torch.Size([1, 1, 2])
torch.Size([5, 1, 2, 2])


In [479]:
sig

tensor([[[0.5249],
         [0.4072]]], dtype=torch.float64)

In [136]:
x_batch @ S_t_s @ x_trans

tensor([[[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]],

        [[1.7079]]])

In [138]:
x_trans  @  x_batch @ S_t_s

tensor([[[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 2.2763, -0.5684]],

        [[ 2.2763, -0.5684],
         [ 

In [2141]:
x_trans @ x_batch @ S_t_T.transpose(0,1)

tensor([[[[-2.3787e+00,  1.9878e+00],
          [-2.3787e+00,  1.9878e+00]],

         [[-2.3603e+00,  1.9828e+00],
          [-2.3603e+00,  1.9828e+00]],

         [[-2.3420e+00,  1.9778e+00],
          [-2.3420e+00,  1.9778e+00]],

         [[-2.3236e+00,  1.9728e+00],
          [-2.3236e+00,  1.9728e+00]],

         [[-2.3053e+00,  1.9677e+00],
          [-2.3053e+00,  1.9677e+00]],

         [[-2.2870e+00,  1.9627e+00],
          [-2.2870e+00,  1.9627e+00]],

         [[-2.2687e+00,  1.9577e+00],
          [-2.2687e+00,  1.9577e+00]],

         [[-2.2504e+00,  1.9527e+00],
          [-2.2504e+00,  1.9527e+00]],

         [[-2.2321e+00,  1.9477e+00],
          [-2.2321e+00,  1.9477e+00]],

         [[-2.2139e+00,  1.9427e+00],
          [-2.2139e+00,  1.9427e+00]],

         [[-2.1956e+00,  1.9377e+00],
          [-2.1956e+00,  1.9377e+00]],

         [[-2.1774e+00,  1.9327e+00],
          [-2.1774e+00,  1.9327e+00]],

         [[-2.1592e+00,  1.9277e+00],
          [-2.1592e+00,  1

In [2143]:
sig = torch.tensor([[[0.5249],[0.4072]]], dtype=torch.float32) 

In [140]:
traces = torch.einsum('bcii->bc', sig @ sig.transpose(1,2) @ S_t_T.transpose(0,1)).unsqueeze(1).unsqueeze(2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [2147]:
time_grids

tensor([[0.1000, 0.2800, 0.4600, 0.6400, 0.8200, 1.0000],
        [0.1053, 0.2842, 0.4632, 0.6421, 0.8211, 1.0000],
        [0.1105, 0.2884, 0.4663, 0.6442, 0.8221, 1.0000],
        [0.1158, 0.2926, 0.4695, 0.6463, 0.8232, 1.0000],
        [0.1211, 0.2968, 0.4726, 0.6484, 0.8242, 1.0000],
        [0.1263, 0.3011, 0.4758, 0.6505, 0.8253, 1.0000],
        [0.1316, 0.3053, 0.4789, 0.6526, 0.8263, 1.0000],
        [0.1368, 0.3095, 0.4821, 0.6547, 0.8274, 1.0000],
        [0.1421, 0.3137, 0.4853, 0.6568, 0.8284, 1.0000],
        [0.1474, 0.3179, 0.4884, 0.6589, 0.8295, 1.0000],
        [0.1526, 0.3221, 0.4916, 0.6611, 0.8305, 1.0000],
        [0.1579, 0.3263, 0.4947, 0.6632, 0.8316, 1.0000],
        [0.1632, 0.3305, 0.4979, 0.6653, 0.8326, 1.0000],
        [0.1684, 0.3347, 0.5011, 0.6674, 0.8337, 1.0000],
        [0.1737, 0.3389, 0.5042, 0.6695, 0.8347, 1.0000],
        [0.1789, 0.3432, 0.5074, 0.6716, 0.8358, 1.0000],
        [0.1842, 0.3474, 0.5105, 0.6737, 0.8368, 1.0000],
        [0.189

In [2149]:
dts = time_grids[:,1:] - time_grids[:,:-1]

In [2151]:
dts.shape

torch.Size([20, 5])

In [2153]:
dts

tensor([[0.1800, 0.1800, 0.1800, 0.1800, 0.1800],
        [0.1789, 0.1789, 0.1789, 0.1789, 0.1789],
        [0.1779, 0.1779, 0.1779, 0.1779, 0.1779],
        [0.1768, 0.1768, 0.1768, 0.1768, 0.1768],
        [0.1758, 0.1758, 0.1758, 0.1758, 0.1758],
        [0.1747, 0.1747, 0.1747, 0.1747, 0.1747],
        [0.1737, 0.1737, 0.1737, 0.1737, 0.1737],
        [0.1726, 0.1726, 0.1726, 0.1726, 0.1726],
        [0.1716, 0.1716, 0.1716, 0.1716, 0.1716],
        [0.1705, 0.1705, 0.1705, 0.1705, 0.1705],
        [0.1695, 0.1695, 0.1695, 0.1695, 0.1695],
        [0.1684, 0.1684, 0.1684, 0.1684, 0.1684],
        [0.1674, 0.1674, 0.1674, 0.1674, 0.1674],
        [0.1663, 0.1663, 0.1663, 0.1663, 0.1663],
        [0.1653, 0.1653, 0.1653, 0.1653, 0.1653],
        [0.1642, 0.1642, 0.1642, 0.1642, 0.1642],
        [0.1632, 0.1632, 0.1632, 0.1632, 0.1632],
        [0.1621, 0.1621, 0.1621, 0.1621, 0.1621],
        [0.1611, 0.1611, 0.1611, 0.1611, 0.1611],
        [0.1600, 0.1600, 0.1600, 0.1600, 0.1600]])

In [2155]:
traces

tensor([[[[-0.1760, -0.1729, -0.1697, -0.1666, -0.1634, -0.1603, -0.1572,
           -0.1540, -0.1509, -0.1478, -0.1446, -0.1415, -0.1383, -0.1352,
           -0.1321, -0.1289, -0.1258, -0.1227, -0.1195, -0.1164]]],


        [[[-0.0434, -0.0411, -0.0389, -0.0366, -0.0343, -0.0320, -0.0297,
           -0.0273, -0.0250, -0.0227, -0.0204, -0.0181, -0.0158, -0.0134,
           -0.0111, -0.0088, -0.0064, -0.0041, -0.0018,  0.0006]]],


        [[[ 0.0827,  0.0845,  0.0862,  0.0879,  0.0897,  0.0914,  0.0931,
            0.0949,  0.0966,  0.0984,  0.1001,  0.1019,  0.1037,  0.1054,
            0.1072,  0.1090,  0.1108,  0.1126,  0.1143,  0.1161]]],


        [[[ 0.2240,  0.2253,  0.2266,  0.2278,  0.2291,  0.2304,  0.2316,
            0.2329,  0.2342,  0.2355,  0.2367,  0.2380,  0.2393,  0.2405,
            0.2418,  0.2431,  0.2444,  0.2456,  0.2469,  0.2482]]],


        [[[ 0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,
            0.4413,  0.4413,  0.4413,  0.4413,  0.4413

In [2157]:
dts

tensor([[0.1800, 0.1800, 0.1800, 0.1800, 0.1800],
        [0.1789, 0.1789, 0.1789, 0.1789, 0.1789],
        [0.1779, 0.1779, 0.1779, 0.1779, 0.1779],
        [0.1768, 0.1768, 0.1768, 0.1768, 0.1768],
        [0.1758, 0.1758, 0.1758, 0.1758, 0.1758],
        [0.1747, 0.1747, 0.1747, 0.1747, 0.1747],
        [0.1737, 0.1737, 0.1737, 0.1737, 0.1737],
        [0.1726, 0.1726, 0.1726, 0.1726, 0.1726],
        [0.1716, 0.1716, 0.1716, 0.1716, 0.1716],
        [0.1705, 0.1705, 0.1705, 0.1705, 0.1705],
        [0.1695, 0.1695, 0.1695, 0.1695, 0.1695],
        [0.1684, 0.1684, 0.1684, 0.1684, 0.1684],
        [0.1674, 0.1674, 0.1674, 0.1674, 0.1674],
        [0.1663, 0.1663, 0.1663, 0.1663, 0.1663],
        [0.1653, 0.1653, 0.1653, 0.1653, 0.1653],
        [0.1642, 0.1642, 0.1642, 0.1642, 0.1642],
        [0.1632, 0.1632, 0.1632, 0.1632, 0.1632],
        [0.1621, 0.1621, 0.1621, 0.1621, 0.1621],
        [0.1611, 0.1611, 0.1611, 0.1611, 0.1611],
        [0.1600, 0.1600, 0.1600, 0.1600, 0.1600]])

In [2159]:
Tr_squeezed.squeeze()

tensor([[-0.1395, -0.1367, -0.1339, -0.1311, -0.1284, -0.1256, -0.1228, -0.1200,
         -0.1173, -0.1145, -0.1117, -0.1089, -0.1061, -0.1033, -0.1005, -0.0977,
         -0.0949, -0.0921, -0.0893, -0.0865],
        [ 0.0121,  0.0141,  0.0161,  0.0181,  0.0201,  0.0221,  0.0241,  0.0261,
          0.0281,  0.0301,  0.0321,  0.0342,  0.0362,  0.0383,  0.0403,  0.0424,
          0.0444,  0.0465,  0.0485,  0.0506],
        [ 0.1697,  0.1713,  0.1729,  0.1745,  0.1760,  0.1776,  0.1792,  0.1808,
          0.1824,  0.1840,  0.1856,  0.1872,  0.1888,  0.1903,  0.1919,  0.1935,
          0.1951,  0.1967,  0.1983,  0.1999],
        [ 0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,
          0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,
          0.4413,  0.4413,  0.4413,  0.4413]])

In [2161]:
traces.squeeze() 

tensor([[-0.1760, -0.1729, -0.1697, -0.1666, -0.1634, -0.1603, -0.1572, -0.1540,
         -0.1509, -0.1478, -0.1446, -0.1415, -0.1383, -0.1352, -0.1321, -0.1289,
         -0.1258, -0.1227, -0.1195, -0.1164],
        [-0.0434, -0.0411, -0.0389, -0.0366, -0.0343, -0.0320, -0.0297, -0.0273,
         -0.0250, -0.0227, -0.0204, -0.0181, -0.0158, -0.0134, -0.0111, -0.0088,
         -0.0064, -0.0041, -0.0018,  0.0006],
        [ 0.0827,  0.0845,  0.0862,  0.0879,  0.0897,  0.0914,  0.0931,  0.0949,
          0.0966,  0.0984,  0.1001,  0.1019,  0.1037,  0.1054,  0.1072,  0.1090,
          0.1108,  0.1126,  0.1143,  0.1161],
        [ 0.2240,  0.2253,  0.2266,  0.2278,  0.2291,  0.2304,  0.2316,  0.2329,
          0.2342,  0.2355,  0.2367,  0.2380,  0.2393,  0.2405,  0.2418,  0.2431,
          0.2444,  0.2456,  0.2469,  0.2482],
        [ 0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,
          0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,  0.4413,
      

In [2163]:
Tr_squeezed = traces.squeeze()  # 形状变为 [4]
#dt_transposed = dts.t()  # 由于dt的原始形状为[1, 4]，这里使用t()进行转置以匹配形状[4, 1]
Tr_squeezed #= (Tr_squeezed[:-1] + Tr_squeezed[1:]) / 2
# 计算矩阵乘法
v_tx =  dts@Tr_squeezed
print(dts.shape)

torch.Size([20, 5])


In [2165]:
torch.diag(v_tx).squeeze()

tensor([0.0952, 0.0961, 0.0970, 0.0980, 0.0989, 0.0997, 0.1006, 0.1015, 0.1023,
        0.1031, 0.1039, 0.1047, 0.1055, 0.1062, 0.1070, 0.1077, 0.1084, 0.1091,
        0.1097, 0.1104])

In [2167]:
(x_batch @ S_t_s @ x_trans).squeeze()

tensor([-1.0388, -1.0200, -1.0013, -0.9826, -0.9640, -0.9453, -0.9268, -0.9082,
        -0.8897, -0.8713, -0.8528, -0.8345, -0.8161, -0.7978, -0.7795, -0.7613,
        -0.7431, -0.7249, -0.7068, -0.6887])

In [2169]:

v_tx = torch.diag(v_tx).squeeze() + (x_batch @ S_t_s @ x_trans).squeeze()

In [2171]:
v_tx

tensor([-0.9436, -0.9239, -0.9042, -0.8846, -0.8651, -0.8456, -0.8262, -0.8068,
        -0.7874, -0.7682, -0.7489, -0.7298, -0.7106, -0.6916, -0.6726, -0.6536,
        -0.6347, -0.6159, -0.5971, -0.5784])

In [76]:
S_repl

tensor([[[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]],


        [[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]],


        [[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]],


        [[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]],


        [[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
      

In [ ]:
S_t_s

In [122]:
H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.float32)
M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.float32)
sigma = torch.tensor([[0.5], [0.5]], dtype=torch.float32)  
C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive semi-definite
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive definite
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive semi-definite
T = 5.0

def euler_step(S, dt):
    
    dS = -2 * H.T @ S + S @ M @ torch.inverse(D) @ M.T @ S - C

    dt_resized = dt[:, None, None]

    S_next = S + dS * dt_resized

    return S_next


In [ ]:
def solve_riccati_ode(self, time_grid):

    if not isinstance(time_grid, torch.Tensor):
        raise TypeError("time_grid should be an batch_size*1 torch.Tensor")
    else:
        if not ((np.abs(time_grid[-1] - self.T) <= 1e-12) or (time_grid[0]>=0)):
            raise Exception("Please ensure that the first entry of time_grid is 0 and the last entry is equal to T.")
        else:
            time_grid_in = time_grid

    time_grid_in = torch.flip(time_grid_in, dims=[0])
    S = self.R.clone()
    S_list = []
    S_list.append(S)

    for i in range(1, len(time_grid_in)):
        dt = time_grid_in[i] - time_grid_in[i-1]
        #Notice that here every dt is a negative value for backward scheme.
        if self.method == "euler":
            S = self.euler_step(S, dt)
        elif self.method == "rk4":
            S = self.rk4_step(S, dt)
        else:
            raise ValueError("Unsupported method")
        S_list.append(S)

    return torch.stack(S_list[::-1])

def euler_step(S, dt):
    dS = -2 * self.H.T @ S + S @ self.M @ torch.inverse(self.D) @ self.M.T @ S - self.C
    return S + dS * dt

def rk4_step(self, S, dt):
    def riccati_derivative(S):
        return -2 * self.H.T @ S + S @ self.M @ torch.inverse(self.D) @ self.M.T @ S - self.C
    k1 = riccati_derivative(S)
    k2 = riccati_derivative(S + 0.5 * dt * k1)
    k3 = riccati_derivative(S + 0.5 * dt * k2)
    k4 = riccati_derivative(S + dt * k3)

    return S + (k1 + 2*k2 + 2*k3 + k4) * dt / 6